<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BorutaSHAP" data-toc-modified-id="BorutaSHAP-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BorutaSHAP</a></span></li><li><span><a href="#Suggested-Example-Workflow" data-toc-modified-id="Suggested-Example-Workflow-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Suggested Example Workflow</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Deal-with-potential-correlation-/-multicollinearity-first." data-toc-modified-id="Step-1:-Deal-with-potential-correlation-/-multicollinearity-first.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Step 1: Deal with potential correlation / multicollinearity first.</a></span></li><li><span><a href="#Step-2:-Run-BorutaSHAP" data-toc-modified-id="Step-2:-Run-BorutaSHAP-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Step 2: Run BorutaSHAP</a></span></li><li><span><a href="#Step-3:-Fit-final-model-with-these-features" data-toc-modified-id="Step-3:-Fit-final-model-with-these-features-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Step 3: Fit final model with these features</a></span></li><li><span><a href="#Step-4:-Explain-using-SHAP-(or-another-method)" data-toc-modified-id="Step-4:-Explain-using-SHAP-(or-another-method)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Step 4: Explain using SHAP (or another method)</a></span></li></ul></li></ul></div>

In [ ]:
from BorutaShap import BorutaShap
from BorutaShap import load_data

from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import ml_inspector # From https://gitlab.nist.gov/gitlab/nam4/ml_inspector
import pandas as pd
import numpy as np

from hyperopt import hp
import hpsklearn
from hpsklearn import HyperoptEstimator
from hyperopt import tpe

In [ ]:
%load_ext watermark
%watermark -t -m -h -v --iversions

# BorutaSHAP

In [ ]:
selector = BorutaShap(model=None, # Defaults to sklearn RandomForest() and all its defaults - no RNG seed so not reproducible
                      importance_measure='shap', # Can be 'gini' or 'shap'
                      classification=False, # False for regression problems
                      percentile=100, pvalue=0.05 # These are default values to behave like the original R package
                     )

# If 'shap': uses shap.TreeExplainer(self.model, feature_perturbation = "tree_path_dependent")

In [ ]:
# BorutaSHAP comes pre-loaded with some data as an example
X, y = load_data(data_type='regression')
X.head()

In [ ]:
# Fit the feature selector
# Train vs. test is an open question: https://compstat-lmu.github.io/iml_methods_limitations/pfi-data.html#introduction-to-test-vs.training-data
selector.fit(X=X,  # Must be a Pandas DataFrame
             y=y, 
             n_trials=100, 
             sample=False, # Use KS-test to choose a sample of the data to compute importance values on instead of entire dataset (for speed)
             train_or_test='test', # Do internal 70/30 train/test split
             normalize=True, # Normalize metric (i.e., shap value importance) by (x-x_mean)/x_std
             verbose=True,
             random_state=0 # For reproducibility, but only relevant if model in selector is also specified
            )

# If train_or_test == 'train' model is trained and evaluated on same data (entirely of X provided); however,
# if 'test', a train_test_split with a 70:30 of X using the random_state value when fit() is called (see below).

# The Bonferroni correction for multiple comparisons is used, as in the original R version of Boruta

In [ ]:
selector.plot(which_features='all')

In [ ]:
sorted(zip(selector.columns, selector.shap_values), key=lambda x:x[1], reverse=True)

In [ ]:
selector.Subset().head() # Select just the columns that passed --> this is the data you can then train on

# By default, all features that are weakly supported (tentative) are also included

In [ ]:
selector.tentative

In [ ]:
selector.accepted

# Suggested Example Workflow

In [ ]:
# In production, the following workflow is recommended.

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target

# for y, 0 = Malignant, 1 = Benign 

# See description at: http://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29

## Step 1: Deal with potential correlation / multicollinearity first.

In [ ]:
# Look at multicollinearity
selected_features, cluster_id_to_feature_ids = ml_inspector.data.InspectData.cluster_collinear(X, # Can use entire dataset since this is unsupervised
                                                                              figsize=(12, 8), 
                                                                              display=True, 
                                                                              t=2,
                                                                              feature_names=None) # None returns indices, otherwise can specify: data.feature_names.tolist())

In [ ]:
print('Chose {} out of {} features'.format(len(selected_features), X.shape[1]))

In [ ]:
clf = XGBClassifier(random_state=0, objective='binary:logistic', eval_metric='logloss', seed=0)

In [ ]:
# This replicates the inner workings of BorutaSHAP
X_train_, X_test_, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

# See what happens when all features are used
clf.fit(X_train_, y_train)
clf.score(X_test_, y_test)

In [ ]:
# Fit again just using only decorrelated subset of features
X_train, X_test = X_train_[:,selected_features], X_test_[:,selected_features]
clf.fit(X_train, y_train) 
clf.score(X_test, y_test) # Almost identical to using all features but greatly reduced (8/30)

# Note that we could also be a little more lenient to allow more features to get through, with the possibility
# that they are more correlated.

## Step 2: Run BorutaSHAP

In [ ]:
selector = BorutaShap(model=clf.__class__(**clf.get_params()),
                      importance_measure='shap',
                      classification=True, 
                      percentile=100, pvalue=0.05
                     )

In [ ]:
# Use BorutaSHAP to see which of the decorrelated dimensions carry information
X_decor = pd.DataFrame(data=X[:,selected_features], columns=data.feature_names[selected_features])

selector.fit(X=X_decor, 
             y=y, 
             n_trials=100, 
             sample=False,
             train_or_test='test', # Does internal 70:30 train/test split
             normalize=True,
             verbose=True,
             random_state=0
            )

In [ ]:
selector.plot(which_features='all')

In [ ]:
selector.Subset().head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selector.Subset().values, y, test_size=0.7, random_state=0)
clf.fit(X_train, y_train) 
clf.score(X_test, y_test) 

# All features (93.23%, 30 features) > 
# decorrelated (92.98%, 8 features) > 
# BorutaSHAP(decorrelated) (92.73%, 6 features)
# Overall performance is nearly identical

# However - note that this is dependent upon the model chosen.  sklearn's classifiers, random forest, etc.
# will give different results from these XGBClassifier results.

In [ ]:
# For comparison, see what happens when we don't initially decorrelate, and use all the features
selector_no_decor = BorutaShap(model=clf.__class__(**clf.get_params()), 
                               importance_measure='shap',
                               classification=True, 
                               percentile=100, pvalue=0.05
                              )

selector_no_decor.fit(X=pd.DataFrame(data=X, columns=data.feature_names),
                      y=y, 
                      n_trials=100, 
                      sample=False,
                      train_or_test='test', 
                      normalize=True,
                      verbose=True,
                      random_state=0
                     )

selector_no_decor.plot(which_features='all')

In [ ]:
# Note the fact that when no decorrelation performed, many high ranking features are actually considered correlated
# For example, 'worst concave points' and 'mean concave points' are the 2 leading features here, which are
# "covered" by 'mean compactness' in the decorrelated set; similarly, ('worst perimeter', 'worst area') --> 'mean radius'
for k,v in sorted(cluster_id_to_feature_ids.items(), key=lambda x: x[0]):
    print(k, [data.feature_names[i].upper() if i in selected_features else data.feature_names[i].lower() for i in v])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selector_no_decor.Subset().values, y, test_size=0.7, random_state=0)
clf.fit(X_train, y_train) 
clf.score(X_test, y_test) 

# Slightly improved relative to using all features!
# This sort of behavior has been noted before (https://medium.com/analytics-vidhya/is-this-the-best-feature-selection-algorithm-borutashap-8bc238aa1677)
# The performance is essentially identical, and now it relies on features that are clearly correlated
# and therefore expecting a reasonable explanation for these results will be difficult, e.g.,
# perimeters and areas.

## Step 3: Fit final model with these features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(selector.Subset().values, 
                                                    y, 
                                                    test_size=0.7, # Can be different from BorutaSHAP if desired 
                                                    random_state=0)
clf.fit(X_train, y_train) 
print('{}% test accuracy for {} with hyperparameters: {}'.format('%.2f'%(100.0*clf.score(X_test, y_test)), 
                                                                 clf.__class__,
                                                                 clf.get_params()
                                                                ))

In [ ]:
# In reality, we would actually perform some optimization perhaps with auto-sklearn or hyperopt-sklearn
# to optimize the final model. These pipelines may include regularization to economize the list of features
# from BorutaSHAP, which only test if they carry information that can be used to make a prediction.

# The following example uses hyperopt-sklearn

In [ ]:
# see help(hpsklearn.components._xgboost_hp_space)
kwargs={'n_jobs':-1,
        'random_state':0, # For reproducibility
        'n_estimators':hp.uniformint('n_estimators', 1, 100),
        'learning_rate':hp.loguniform('learning_rate', low=np.log(1e-5), high=np.log(1e-2)),
        'max_depth':hp.uniformint('max_depth', 3, 10),
       }

# Instantiate a HyperoptEstimator with the search space and number of evaluations
opt_estim = HyperoptEstimator(classifier=hpsklearn.components.xgboost_classification('XGBClassifier', **kwargs), 
                          preprocessing=[], # Trees don't need any scaling -NOTE: "none" = random choice not "don't use any"
                          algo=tpe.suggest, # Use tree parzen estimator
                          max_evals=100, # Total number of models to evaluate - USE MORE IN PRACTICE
                          trial_timeout=10, # Allow this many seconds to fit each - USE LONGER IN PRACTICE
                          seed=0, # For reproducibility
                          refit=True) # Refit the best model on entire training set when done

# Search the hyperparameter space based on the data
opt_estim.fit(X=X_train, y=y_train, 
              valid_size=0.0, # Don't need separate validation set, use all of training
              n_folds=3,  # Stratified by default - CAN USE MORE IN PRACTICE
              cv_shuffle=True,
              random_state=0) 

In [ ]:
# These results might seem lower than the simple, first pass - however, this is a cross-validated result.

# This is automatically done by hyperopt-sklearn, but just make sure the model is retrained on all of the 
# training data, not just k-1 folds.
model = opt_estim.best_model()['learner']
model.fit(X_train, y_train)

print('{}% test accuracy, {}% train accuracy'.format('%.2f'%(100.0*opt_estim.score(X_test, y_test)), 
                                                     '%.2f'%(100.0*opt_estim.score(X_train, y_train))
                                                    )
     )

## Step 4: Explain using SHAP (or another method)

In [ ]:
# The selector above was fit with shadow features so the result will not be identical to this.
# Here, we perform the SHAP analysis again but now with the final model.
# Note: we can use more stringent samping parameters, etc. here since we don't have to iterate as we
# have to in BorutaSHAP; this means we can be more accurate about the SHAP values at this stage.

In [ ]:
import shap
shap.initjs()

In [ ]:
# We can use the TreeExplainer since we have a tree-based model, or the more general KernelExplainer.
# Both produce very similar results using the settings below.

# Note: had we predicted log-odds for this binary classification task we would use 'logit' link functions
# in the below examples and its 'raw' output.  Instead, we will analyze the output of predict_proba()

use_tree = True

if use_tree:
    # if model output = raw it seems to just expaine the last column of the model prediction (class prob here)
    explainer = shap.TreeExplainer(model=model,
                                   model_output='predict_proba',
                                   data=X_train
                                  )
    shap_values = explainer.shap_values(X_test, check_additivity=True) # The tree explainer can be checked for additivity (shap + prediction = mean)
    # It seems that trying to do this explicitly later is problematic, apparently because of the way TreeExplainer works.
else:
    explainer = shap.KernelExplainer(model.predict_proba,
                                     data=X_train, 
                                     link='identity') 
    shap_values = explainer.shap_values(X_test, nsamples=100)
    # Additivity can be explicitly checked later when using the KernelExplainer

In [ ]:
# Note that these probabilities are clustered near 50% in this model so we are not explaining a large number - this is ok
for i in [0,1]:
    probs = model.predict_proba(X_test)[model.predict(X_test) == i][:,i]
    mean, std= np.mean(probs), np.std(probs)
    print('When i={}, mean(Prob)={} +/- {}'.format(i, mean, std))

In [ ]:
# It turns out that classes "0" and "1" are "in order" but this is not guaranteed
print('Model stored classes as: {}'.format(model.classes_))

# Probability is predicted for each class - since binary we can just look at one - CHECK WHAT THIS MEANS
examine_class_idx = 0 

print('Looking at class {} which means {}'.format(model.classes_[examine_class_idx],
                                                  'malignant' if model.classes_[examine_class_idx] == 0 else 'benign'
                                                 ))

In [ ]:
# Overall summary of average effect of each feature
shap.summary_plot(shap_values, #[examine_class_idx], 
                  X_test, 
                  plot_type="bar",
                  feature_names=selector.Subset().columns
                 )

In [ ]:
# A jittered violin plot is more helpful to summarize trends in the sign of SHAP value vs. prediction
shap.summary_plot(shap_values[examine_class_idx], 
                  X_test, 
                  feature_names=selector.Subset().columns
                 )

In [ ]:
# Look at decision paths
misclassified = y_test != model.predict(X_test)

shap.decision_plot(explainer.expected_value[examine_class_idx], 
                   shap_values[examine_class_idx], 
                   feature_names=selector.Subset().columns.tolist(),
                   link='identity',
                   feature_order='importance',
                   highlight=misclassified
                   )

In [ ]:
# Plot the misclassified ones
shap.decision_plot(explainer.expected_value[examine_class_idx], 
                   shap_values[examine_class_idx][misclassified,:], 
                   feature_names=selector.Subset().columns.tolist(),
                   link='identity',
                   feature_order='importance',
                   title='ONLY THE MISCLASSIFIED DATA POINTS',
                   )

In [ ]:
# See here for visualizing multioutput decision plots, which are useful when we have more than 2 classes:
# https://slundberg.github.io/shap/notebooks/plots/decision_plot.html

In [ ]:
shap.force_plot(explainer.expected_value[examine_class_idx], 
                shap_values[examine_class_idx], 
                X_test, 
                link='identity',
                feature_names=selector.Subset().columns
               )

In [ ]:
# Examine an individual instance
instance_idx = 123

print('Correct answer = {}, predicted probabilities = {}'.format(y_test[instance_idx], 
                                                   model.predict_proba(X_test[instance_idx,:].reshape(1,-1))
                                                  ))
if not use_tree:
    print('\nThe SHAP values for this instance are:')
    for i in range(shap_values[examine_class_idx].shape[1]):
        print('{} = {}'.format(selector.Subset().columns[i], shap_values[examine_class_idx][instance_idx,i]))

    print('\nThese sum to {}'.format(np.sum(shap_values[examine_class_idx][instance_idx,:])))
    pred_ = model.predict_proba(X_test[instance_idx,:].reshape(1,-1))[0][examine_class_idx]
    # mean_ = model.predict_proba(X_train)[:,examine_class_idx])
    mean_ = explainer.expected_value[examine_class_idx]
    print('\nIndeed, this should be the prediction - mean (from training set) = {} - {} = {}'.format(pred_,
                                                                                                   mean_,
                                                                                                   pred_ -mean_
                                                                                                  ))
    assert(np.abs((pred_ - mean_) - np.sum(shap_values[examine_class_idx][instance_idx,:])) < 1.0e-8)

shap.force_plot(explainer.expected_value[examine_class_idx], 
                shap_values[examine_class_idx][instance_idx,:], 
                X_test[instance_idx,:], 
                link='identity',
                feature_names=selector.Subset().columns
               )

In [ ]:
# Vertical dispersion at a single value represents interaction effects with other features

In [ ]:
# Mean radius itself increases the probability of this class index if > 16 and decreases it if < 16
shap.dependence_plot('mean radius', 
                     shap_values=shap_values[examine_class_idx], 
                     features=pd.DataFrame(data=X_test, columns=selector.Subset().columns)
                    )

In [ ]:
# Mean texture itself increases the probability of this class index if > 20 and decreases it if < 16

# Overall the mean texture has less of an impact if the mean compactness is also low.
shap.dependence_plot('mean texture', 
                     shap_values=shap_values[examine_class_idx], 
                     features=pd.DataFrame(data=X_test, columns=selector.Subset().columns)
                    )

In [ ]:
# Mean compactness itself increases the probability of this class index if > .12 and decreases it if < .12,
# though there appear to be essentially 3 different "levels"

# Overall the mean compactness (perimeter^2 / area - 1.0) has less of an impact if the radius error is high
shap.dependence_plot('mean compactness', 
                     shap_values=shap_values[examine_class_idx], 
                     features=pd.DataFrame(data=X_test, columns=selector.Subset().columns)
                    )

In [ ]:
# Mean smoothness itself increases the probability of this class index if > .09 and decreases it if < .09,
# though there appear to be essentially 3 different "levels"
shap.dependence_plot('mean smoothness', 
                     shap_values=shap_values[examine_class_idx], 
                     features=pd.DataFrame(data=X_test, columns=selector.Subset().columns)
                    )

In [ ]:
# Very little effect of texture error
shap.dependence_plot('texture error', 
                     shap_values=shap_values[examine_class_idx], 
                     features=pd.DataFrame(data=X_test, columns=selector.Subset().columns)
                    )